# Timelag Differential Equation Tutorial

The purpose of this notebook is to demonstrate the use of the physics-based fuel moisture model used with `wrfxpy`. This model is used as a benchmark for the machine learning methods. The model is a time-lag ODE plus a Kalman filter for assimilating fuel moisture observations.

## Setup

In [ ]:
import os.path as osp
import json
import sys
import numpy as np
import pandas as pd
sys.path.append('../src')
from utils import Dict, read_yml, read_pkl, str2time, print_dict_summary, time_range, rename_dict
from data_funcs import retrieve_fmda_data
import models.moisture_models as mm
import matplotlib.pyplot as plt

In [ ]:
params = read_yml("../etc/params_models.yaml", subkey="ode")
print_dict_summary(params)

In [ ]:
dat = read_pkl("../data/test.pkl")

In [ ]:
def is_consecutive_hours(times):
    # Convert to numpy timedelta64[h] for hour differences
    time_diffs = np.diff(times).astype('timedelta64[h]')
    return np.all(time_diffs == np.timedelta64(1, 'h'))

In [ ]:
print(f"Total hours: {dat["BRLW4"]["times"].shape}")
print(f"Check consecutive times: {is_consecutive_hours(dat["BRLW4"]["times"])}")

In [ ]:
ode = mm.ODE_FMC()

In [ ]:
m, errs = ode.run_model(dat, hours=72, h2=48)

In [ ]:
m.shape

In [ ]:
errs

In [ ]:
plot_styles = {
    'fm': {'color': '#468a29', 'linestyle':"-", 'label':'FM Observed'},
    'preds': {'color': 'k', 'linestyle':"-", 'label':'FM Model'},
    'Ed': {'color': '#EF847C', 'linestyle': '--', 'alpha':.8, 'label': 'drying EQ'},
    'Ew': {'color': '#7CCCEF', 'linestyle': '--', 'alpha':.8, 'label': 'wetting EQ'},
    'rain': {'color': 'b', 'linestyle': '-', 'alpha':.9, 'label': 'Rain'}
}

In [ ]:
def plot_feature(x, y, feature_name):
    style = plot_styles.get(feature_name, {})
    plt.plot(x, y, **style)

In [ ]:
import matplotlib.dates as mdates

In [ ]:
def plot_fmc(times, m, title="", time_format = mdates.DateFormatter('%Y-%m-%d, %H:%M:%S')):
    fig, ax = plt.subplots()
    style = plot_styles.get("fm", {})
    ax.plot(times, m, **style)
    ax.xaxis.set_major_formatter(time_format)
    
    # plot_feature(times, m, "fm")
    plt.xticks(rotation=90)
    if not title == "":
        plt.title(title)

In [ ]:
dat["VRFN1"]["RAWS"]["fm"]

In [ ]:
plot_feature(dat["VRFN1"]["times"], dat["VRFN1"]["RAWS"]["fm"], "fm")

In [ ]:
plot_fmc(dat["VRFN1"]["times"], dat["VRFN1"]["RAWS"]["fm"])

In [ ]:
type(dat["VRFN1"]["times"][0])